# COGS 118B Final Project
## Rhythm Game Beat Chart Difficulty Classification

**Contributers:** Miles Davis, Theo Bui, Alan De Luna, Sanjith Shanmugavel, Nicole Huynh

#### Introduction

Rhythm games are blah blah blah. Add relevant background and motivation for classifying difficulties. Maybe something along the lines of: "Difficulty levels between songs in ___ game are inconsistent. Some beat charts may be considered more difficult because of the number of required hits for a high score simply increasing, increasing tempo, or rhythms becoming more complex."

#### Algorithm Choice
???

#### Data Preprocessing
1. read in `.tja` files as string
2. convert string to `numpy` array
3. extract relevant information

Metadata:
- `BPM`
- `OFFSET`
- `GENRE`

Course info:
- `COURSE`
- `LEVEL`
- `BALLOON`
- song notation -- written between `#START` and `#END` commands
- timing notation -- `#MEASURE` and `#BPMCHANGE`
    - from Taiko explanation: 'Measures in the chart are separated with a comma character , followed by a line break. Timing between each measure is the same as long as #MEASURE and #BPMCHANGE commands are not used. Measures may contain any amount of notes, including zero, the less numbers there are in a measure, the more far apart the notes will be in the chart, each measure is equally divided by the amount of numbers there are inside. "12," can be written as "1020," and "10002000,", the timing is identical in all three examples.'
- delay -- offsets position of following notation (can be negative to indicate overlap) and written as `#DELAY`
- scroll speed -- `#SCROLL`
- Go-Go Time -- written between `#GOGOSTART` and `#GOGOEND`
- measure lines turned on/off -- `#BARLINEOFF` and `#BARLINEON`
- branching and paths -- ???
- BPM change -- ???

In [26]:
import numpy as np
import pandas as pd
import os

In [27]:
test_path = 'test_data/cruel_angels_thesis.tja'

try:
    with open(test_path, 'r', encoding='utf-8') as file:
        test_str = file.read()
except FileNotFoundError:
    print('file not found :-(')
except Exception as e:
    print(f'an error occurred: {e}')

In [28]:
print(test_str)

﻿//TJADB Project
TITLE:A Cruel Angel's Thesis -New Audio-
TITLEJA:残酷な天使のテーゼ -新曲-
SUBTITLE:--Youko Takahashi/Neon Genesis Evangelion
BPM:80
WAVE:A Cruel Angels Thesis -New Audio-.ogg
OFFSET:-5.041
DEMOSTART:68.079

COURSE:Edit
LEVEL:8
SCOREINIT:740
SCOREDIFF:208


#START


,
#BPMCHANGE 79
1000100010010010,
#MEASURE 8/4
#BPMCHANGE 76.7
100000000000100000000000100000000000500000000000
#BPMCHANGE 134
000000000000000000000000000000000008000000000000,
#MEASURE 4/4
#BPMCHANGE 129
1010210121012010,
1010210121020000,
1010210121012010,
1010221030040000,
1010201012102000,
1010201012102000,
1120212011202120,
1120221010000000,
1120201010222000,
1120201010222000,
1020102010222010,
1020102022102210,
1020102010221020,
1020102010221020,
1120221011202210,
1120221010222000,
1010221022221000,
1010221022221000,
1020122010201220,
1101210121012102,
1101102012022020,
1120221011202210,
1101201011012010,
1212,
1101102011011020,
1220122011122020,
1121102011211010,
3000000030030030,

#GOGOSTART
#SECTION
#BRANCHST

In [29]:
# split string by lines
chart = test_str.split('COURSE:')
len(chart)

6

In [30]:
chart

["\ufeff//TJADB Project\nTITLE:A Cruel Angel's Thesis -New Audio-\nTITLEJA:残酷な天使のテーゼ -新曲-\nSUBTITLE:--Youko Takahashi/Neon Genesis Evangelion\nBPM:80\nWAVE:A Cruel Angels Thesis -New Audio-.ogg\nOFFSET:-5.041\nDEMOSTART:68.079\n\n",
 'Edit\nLEVEL:8\nSCOREINIT:740\nSCOREDIFF:208\n\n\n#START\n\n\n,\n#BPMCHANGE 79\n1000100010010010,\n#MEASURE 8/4\n#BPMCHANGE 76.7\n100000000000100000000000100000000000500000000000\n#BPMCHANGE 134\n000000000000000000000000000000000008000000000000,\n#MEASURE 4/4\n#BPMCHANGE 129\n1010210121012010,\n1010210121020000,\n1010210121012010,\n1010221030040000,\n1010201012102000,\n1010201012102000,\n1120212011202120,\n1120221010000000,\n1120201010222000,\n1120201010222000,\n1020102010222010,\n1020102022102210,\n1020102010221020,\n1020102010221020,\n1120221011202210,\n1120221010222000,\n1010221022221000,\n1010221022221000,\n1020122010201220,\n1101210121012102,\n1101102012022020,\n1120221011202210,\n1101201011012010,\n1212,\n1101102011011020,\n1220122011122020,\n1121102

In [31]:
# try to grab just relevant info listed above

# metadata for entire song chart
metadata = chart[0].split('\n')
bpm = int(metadata[4][4:])
offset = float(metadata[6][7:])

# courses by difficulty
edit = [i for i in chart if 'Edit' in i]
oni = [i for i in chart if 'Oni' in i]
hard = [i for i in chart if 'Hard' in i]
normal = [i for i in chart if 'Normal' in i]
easy = [i for i in chart if 'Easy' in i]

In [32]:
course_diff = {'Easy': 0, 'Normal': 1, 'Hard': 2, 'Oni': 3, 'Edit': 4, 'Tower': 5, 'Dan': 6}
lvl_diff = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [33]:
int([i[6:] for i in normal[0].split('\n') if 'LEVEL' in i][0])

4

In [34]:
normal

['Normal\nLEVEL:4\nSCOREINIT:1310\nSCOREDIFF:420\n\n\n#START\n\n\n1,\n#BPMCHANGE 79\n1,\n#BPMCHANGE 76.7\n#MEASURE 8/4\n100000000000100000000000500000000000000000000000\n#BPMCHANGE 134\n000000000000000000000008000000000000000000000000,\n#MEASURE 4/4\n#BPMCHANGE 129\n1110,\n11102000,\n1110,\n1010100030030000,\n1120,\n1120,\n11,\n12,\n10011020,\n10022000,\n10011020,\n10022000,\n1120,\n1120,\n11,\n12,\n10011020,\n10022000,\n10011020,\n500000000000000000000000000008000000000000000000,\n1001000010000000,\n2002000020000000,\n1001000010000000,\n1,\n1001000010000000,\n2002000020000000,\n1001000010000000,\n500000000000000000000000000008000000000000000000,\n\n#GOGOSTART\n1000100010010000,\n1000000020020000,\n1110,\n1000100020020000,\n1000100010010000,\n1000000020020000,\n1110,\n1000100030030000,\n1000100010010000,\n1000000020020000,\n1110,\n1000100030030000,\n\n#GOGOEND\n\n\n#END\n\n\n']

In [35]:
# complete course and measures
normal_full = normal[0].split('#START')[1]
normal_full = normal_full.split('#END')[0].split('\n')
all_measures = [i[:-1] for i in normal_full if ',' in i]

# just go-go time part of course and measures
gogo_time_idx = [
    i for i, val in enumerate(normal_full)
    if val in ('#GOGOSTART', '#GOGOEND')
]
gogo_time = normal_full[gogo_time_idx[0]:gogo_time_idx[1]+1]
gogo_time_measures = [i[:-1] for i in gogo_time if ',' in i]

# just non go-go time part of course and measures
reg_time = normal_full[:gogo_time_idx[0]] + normal_full[gogo_time_idx[1]+1:]
reg_measures = [i[:-1] for i in reg_time if ',' in i]

# every bpm and measure change
# but also taking note of what bpm/time sig the course is being changed to would also be valuable
bpm_change = [i for i in normal_full if '#BPMCHANGE' in i]
measure_change = [i for i in normal_full if '#MEASURE' in i]

# change gogo_time_idx to list of normal_full indices
gogo_time_idx = [i for i in range(gogo_time_idx[0], gogo_time_idx[1]+1)]
reg_time_idx = [i for i in range(len(normal_full)) if i not in gogo_time_idx]

In [36]:
normal_course = {
    'full_chart': normal_full,
    'all_measures': all_measures,
    'full_gogo_chart': gogo_time,
    'gogo_time_measures': gogo_time_measures,
    'gogo_time_indices': gogo_time_idx,
    'full_non_gogo_chart': reg_time,
    'non_gogo_measures': reg_measures,
    'non_gogo_indices': reg_time_idx,
    'all_bpm_changes': bpm_change,
    'all_time_sig_changes': measure_change
}

In [37]:
gogo_time_measures_copy = [i for i in gogo_time_measures]
gogo_time_measures_copy[2] = '1000100010000000'
gogo_time_measures_copy[6] = '1000100010000000'
gogo_time_measures_copy[10] = '1000100010000000'

In [38]:
gogo_time_measures_copy

['1000100010010000',
 '1000000020020000',
 '1000100010000000',
 '1000100020020000',
 '1000100010010000',
 '1000000020020000',
 '1000100010000000',
 '1000100030030000',
 '1000100010010000',
 '1000000020020000',
 '1000100010000000',
 '1000100030030000']

In [39]:
len('1000100010000000')

16

In [40]:
print(np.array(normal_course))

{'full_chart': ['', '', '', '1,', '#BPMCHANGE 79', '1,', '#BPMCHANGE 76.7', '#MEASURE 8/4', '100000000000100000000000500000000000000000000000', '#BPMCHANGE 134', '000000000000000000000008000000000000000000000000,', '#MEASURE 4/4', '#BPMCHANGE 129', '1110,', '11102000,', '1110,', '1010100030030000,', '1120,', '1120,', '11,', '12,', '10011020,', '10022000,', '10011020,', '10022000,', '1120,', '1120,', '11,', '12,', '10011020,', '10022000,', '10011020,', '500000000000000000000000000008000000000000000000,', '1001000010000000,', '2002000020000000,', '1001000010000000,', '1,', '1001000010000000,', '2002000020000000,', '1001000010000000,', '500000000000000000000000000008000000000000000000,', '', '#GOGOSTART', '1000100010010000,', '1000000020020000,', '1110,', '1000100020020000,', '1000100010010000,', '1000000020020000,', '1110,', '1000100030030000,', '1000100010010000,', '1000000020020000,', '1110,', '1000100030030000,', '', '#GOGOEND', '', '', ''], 'all_measures': ['1', '1', '000000000000000

In [41]:
list('1110')

['1', '1', '1', '0']

In [42]:
def format_measures(measures, max_len):
    new_measures = [i for i in measures]
    
    for i in range(len(measures)):
        if len(measures[i]) == 0:
            new_measures[i] = max_len*'0'
        elif len(measures[i]) != max_len:
            zeroes = int((max_len / len(measures[i])) - 1)
            reformat = [i + zeroes*'0' for i in list(measures[i])]
            new_measures[i] = ''.join(reformat)
    
    return new_measures

In [43]:
def handle_branches(chart_lines):
    has_branch = any('#BRANCHSTART' in line for line in chart_lines)
    if not has_branch:
        return chart_lines
    
    result = []
    in_branch = False
    current_branch = None
    branch_content = {'N': [], 'E': [], 'M': []}
    
    for line in chart_lines:
        stripped = line.strip()
        
        if stripped.startswith('#BRANCHSTART'):
            in_branch = True
            branch_content = {'N': [], 'E': [], 'M': []}
            current_branch = None
            continue
            
        elif stripped == '#N':
            current_branch = 'N'
            continue
            
        elif stripped == '#E':
            current_branch = 'E'
            continue
            
        elif stripped == '#M':
            current_branch = 'M'
            continue
            
        elif stripped == '#BRANCHEND':
            # Just picking M if possible and if not cycle through E and then N
            if branch_content['M']:
                result.extend(branch_content['M'])
            elif branch_content['E']:
                result.extend(branch_content['E'])
            elif branch_content['N']:
                result.extend(branch_content['N'])
            
            in_branch = False
            current_branch = None
            continue
        
        if in_branch and current_branch:
            branch_content[current_branch].append(line)
        elif not in_branch:
            result.append(line)
    
    return result

In [54]:
def extract_info(course):
    difficulty = course[0].split('\n')[0]
    level = int([i[6:] for i in course[0].split('\n') if 'LEVEL' in i][0])

    # complete course and measures
    full_course = course[0].split('#START')[1]
    full_course = full_course.split('#END')[0].split('\n')

    full_course = handle_branches(full_course)

    all_measures = [i[:-1] for i in full_course if ',' in i]
    if not all_measures:  # Handle empty measure case
        return None
        
    max_measure_len = len(max(all_measures, key=len))
    all_measures = format_measures(all_measures, max_measure_len)

    # just go-go time part of course and measures
    gogo_time_idx = [
        i for i, val in enumerate(full_course)
        if val in ('#GOGOSTART', '#GOGOEND')
    ]
    
    if len(gogo_time_idx) >= 2:
        gogo_time = full_course[gogo_time_idx[0]:gogo_time_idx[1]+1]
        gogo_time_measures = format_measures([i[:-1] for i in gogo_time if ',' in i], max_measure_len)
        gogo_time_idx_list = [i for i in range(gogo_time_idx[0], gogo_time_idx[1]+1)]
        reg_time_idx = [i for i in range(len(full_course)) if i not in gogo_time_idx_list]
    else:
        gogo_time = []
        gogo_time_measures = []
        gogo_time_idx_list = []
        reg_time_idx = list(range(len(full_course)))

    # just non go-go time part of course and measures
    reg_time = full_course[:gogo_time_idx[0]] + full_course[gogo_time_idx[1]+1:] if len(gogo_time_idx) >= 2 else full_course
    reg_measures = format_measures([i[:-1] for i in reg_time if ',' in i], max_measure_len)

    # every bpm and measure change
    bpm_change = [i for i in full_course if '#BPMCHANGE' in i]
    measure_change = [i for i in full_course if '#MEASURE' in i]

    course_dict = {  # Return plain dict, not numpy array!
        'course_difficulty': difficulty,
        'course_level': level,
        'full_course_chart': full_course,
        'all_measures': all_measures,
        'full_gogo_chart': gogo_time,
        'gogo_time_measures': gogo_time_measures,
        'gogo_time_indices': gogo_time_idx_list,
        'full_non_gogo_chart': reg_time,
        'non_gogo_measures': reg_measures,
        'non_gogo_indices': reg_time_idx,
        'all_bpm_changes': bpm_change,
        'all_time_sig_changes': measure_change
    }

    return course_dict  # Plain dict, no np.array()!

In [61]:
def extract_all_courses(chart):
    metadata = chart[0].split('\n')
    bpm = int([i[4:] for i in metadata if 'BPM:' in i][0])
    offset = float([i[7:] for i in metadata if 'OFFSET:' in i][0])
    
    edit = [i for i in chart if 'Edit' in i]
    oni = [i for i in chart if 'Oni' in i and 'Edit' not in i]  # Avoid matching "Edit" in "Oni"
    hard = [i for i in chart if 'Hard' in i]
    normal = [i for i in chart if 'Normal' in i]
    easy = [i for i in chart if 'Easy' in i]

    all_diff = [easy, normal, hard, oni, edit]
    all_extracted = []

    for d in all_diff:
        if d:  # Only process if difficulty exists
            course_info = extract_info(d)
            all_extracted.append(course_info)

    return all_extracted  # Don't add bpm and offset as list items!

In [46]:
extract_all_courses(chart)

[array({'course_difficulty': 'Easy', 'course_level': 4, 'full_course_chart': ['', '', '', '1,', '#BPMCHANGE 79', '1,', '#BPMCHANGE 76.7', '#MEASURE 8/4', '100000000000100000000000500000000000000000000000', '#BPMCHANGE 134', '000000000000000000000000000000000008000000000000,', '#MEASURE 4/4', '#BPMCHANGE 129', '12,', '1120,', '11,', '1000100030030000,', '11,', '12,', '11,', '12,', '10011000,', '10022000,', '1012,', '10022000,', '11,', '12,', '11,', '12,', '10011000,', '10022000,', '1111,', '500000000000000000000000000000000008000000000000,', '1001000000000000,', '2002000000000000,', '12,', '12,', '1001000000000000,', '2002000000000000,', '11,', '500000000000000000000000000000000008000000000000,', '', '#GOGOSTART', '1110,', '1220,', '1110,', '1120,', '1110,', '1220,', '1110,', '1000000030030000,', '1110,', '1220,', '1110,', '1000000030030000,', '', '#GOGOEND', '', '', ''], 'all_measures': ['100000000000000000000000000000000000000000000000', '1000000000000000000000000000000000000000000000

In [47]:
def extract_note_features(measures):
    """Extract numerical features from note patterns"""
    
    # Concatenate all measures into one sequence
    full_sequence = ''.join(measures)
    
    features = {}
    
    # Basic note counts
    features['total_notes'] = sum(1 for c in full_sequence if c in '1234')
    features['don_notes'] = full_sequence.count('1')  # Red notes (center)
    features['ka_notes'] = full_sequence.count('2')   # Blue notes (rim)
    features['big_don'] = full_sequence.count('3')    # Big red
    features['big_ka'] = full_sequence.count('4')     # Big blue
    features['drumroll'] = full_sequence.count('5')   # Yellow drumrolls
    features['balloon'] = full_sequence.count('6')    # Balloons
    
    # Note density (notes per measure)
    features['note_density'] = features['total_notes'] / len(measures) if measures else 0
    
    # Pattern complexity metrics
    features['note_variety'] = len(set(c for c in full_sequence if c in '123456'))
    
    # Calculate note spacing (average zeros between notes)
    note_positions = [i for i, c in enumerate(full_sequence) if c in '1234']
    if len(note_positions) > 1:
        spacings = [note_positions[i+1] - note_positions[i] 
                   for i in range(len(note_positions)-1)]
        features['avg_spacing'] = np.mean(spacings)
        features['min_spacing'] = np.min(spacings)
        features['spacing_variance'] = np.var(spacings)
    else:
        features['avg_spacing'] = 0
        features['min_spacing'] = 0
        features['spacing_variance'] = 0
    
    # Pattern transitions (how often notes change type)
    transitions = sum(1 for i in range(len(note_positions)-1) 
                     if full_sequence[note_positions[i]] != full_sequence[note_positions[i+1]])
    features['transition_rate'] = transitions / len(note_positions) if note_positions else 0
    
    return features


def extract_timing_features(course_data):
    """Extract features from BPM and timing changes"""
    
    features = {}
    
    # Count timing changes
    features['bpm_changes'] = len(course_data['all_bpm_changes'])
    features['time_sig_changes'] = len(course_data['all_time_sig_changes'])
    
    # Extract BPM values
    bpm_values = []
    for change in course_data['all_bpm_changes']:
        bpm = float(change.split()[1])
        bpm_values.append(bpm)
    
    if bpm_values:
        features['max_bpm'] = max(bpm_values)
        features['min_bpm'] = min(bpm_values)
        features['avg_bpm'] = np.mean(bpm_values)
        features['bpm_variance'] = np.var(bpm_values)
    else:
        features['max_bpm'] = 0
        features['min_bpm'] = 0
        features['avg_bpm'] = 0
        features['bpm_variance'] = 0
    
    return features


def extract_gogo_features(course_data):
    """Extract features from Go-Go Time sections"""
    
    features = {}
    
    gogo_measures = course_data['gogo_time_measures']
    all_measures = course_data['all_measures']
    
    # Proportion of chart in go-go time
    features['gogo_proportion'] = len(gogo_measures) / len(all_measures) if all_measures else 0
    
    # Note density in go-go vs regular sections
    gogo_notes = sum(sum(1 for c in m if c in '1234') for m in gogo_measures)
    regular_measures = course_data['non_gogo_measures']
    regular_notes = sum(sum(1 for c in m if c in '1234') for m in regular_measures)
    
    features['gogo_note_density'] = gogo_notes / len(gogo_measures) if gogo_measures else 0
    features['regular_note_density'] = regular_notes / len(regular_measures) if regular_measures else 0
    
    return features


def create_feature_vector(course_data):
    """Combine all features into a single vector"""
    
    features = {}
    
    # Get all feature types
    note_features = extract_note_features(course_data['all_measures'])
    timing_features = extract_timing_features(course_data)
    gogo_features = extract_gogo_features(course_data)
    
    # Combine all features
    features.update(note_features)
    features.update(timing_features)
    features.update(gogo_features)
    
    # Add the target labels
    features['difficulty_name'] = course_data['course_difficulty']
    features['difficulty_level'] = course_data['course_level']
    
    return features

In [55]:
def process_all_charts(chart_directory):
    """Process all .tja files into a training dataset"""
    
    all_features = []
    
    # Iterate through all .tja files
    for filename in os.listdir(chart_directory):
        if not filename.endswith('.tja'):
            continue
            
        filepath = os.path.join(chart_directory, filename)
        
        try:
            with open(filepath, 'r', encoding='utf-8') as file:
                chart_str = file.read()
            
            # Parse the chart
            chart = chart_str.split('COURSE:')
            
            # Extract all courses
            all_courses = extract_all_courses(chart)
            
            # Process each difficulty
            for course_data in all_courses:
                if course_data is None:  # Skip empty courses
                    continue
                    
                # No need to convert - should already be a dict
                if isinstance(course_data, np.ndarray):
                    course_data = course_data.item()
                
                features = create_feature_vector(course_data)
                features['song_name'] = filename.replace('.tja', '')  # Add song name
                all_features.append(features)
                
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue
    
    # Convert to DataFrame
    df = pd.DataFrame(all_features)
    
    return df

In [49]:
def extract_info_per_measure(chart_lines):
    for line in chart_lines:
        if line.startswith("COURSE:") or line.startswith("LEVEL:"):
            continue

    measures_arr = [
        [],  # 0: notes
        [],  # 1: bpm
        [],  # 2: time signature
        [],  # 3: scroll speed
        [],  # 4: is_gogo
    ]

    current_bpm = 0
    time_sig = (4, 4)
    scroll = 1.0
    is_gogo = False

    inside_chart = False
    current_measure_notes = ""

    for raw_line in chart_lines:
        line = raw_line.strip()

        if not line or line.startswith("//"):
            continue

        # chart boundaries
        if line.startswith("#START"):
            inside_chart = True
            continue

        if line.startswith("#END"):
            inside_chart = False
            break

        if not inside_chart:
            continue

        # measure metadata
        if line.startswith("#BPMCHANGE"):
            current_bpm = float(line.split()[1])
            continue

        if line.startswith("#MEASURE"):
            num, den = line.split()[1].split("/")
            time_sig = (int(num), int(den))
            continue

        if line.startswith("#SCROLL"):
            scroll = float(line.split()[1])
            continue

        if line == "#GOGOSTART":
            is_gogo = True
            continue

        if line == "#GOGOEND":
            is_gogo = False
            continue

        # measure content
        if "," in line:
            note_part = line.replace(",", "")
            current_measure_notes += note_part

            measures_arr[0].append(current_measure_notes)
            measures_arr[1].append(current_bpm)
            measures_arr[2].append(time_sig)
            measures_arr[3].append(scroll)
            measures_arr[4].append(1 if is_gogo else 0)

            current_measure_notes = ""
        else:
            current_measure_notes += line

    return np.array(measures_arr)

In [60]:
df = process_all_charts('test_data/')
print(f"Processed {len(df)} courses")
print(df.head())
df.to_csv('taiko_features.csv', index=False)

Processed 17 courses
   total_notes  don_notes  ka_notes  big_don  big_ka  drumroll  balloon  \
0           88         81         5        2       0         7        0   
1          127         84        26       17       0         4        5   
2          228        142        85        1       0         6        0   
3          462        312       143        5       2         0        0   
4          102         59        22       20       1         2        4   

   note_density  note_variety  avg_spacing  ...  max_bpm  min_bpm  avg_bpm  \
0      1.517241             4    31.034483  ...      0.0      0.0      0.0   
1      2.189655             5    21.523810  ...      0.0      0.0      0.0   
2      3.931034             4    12.000000  ...      0.0      0.0      0.0   
3      7.965517             4     6.026030  ...      0.0      0.0      0.0   
4      1.457143             6    31.782178  ...      0.0      0.0      0.0   

   bpm_variance  gogo_proportion  gogo_note_density  regula

In [59]:
df = process_all_charts('test_data/')
print(f"Processed {len(df)} courses")
print(df[['song_name', 'difficulty_name', 'difficulty_level', 'initial_bpm', 'avg_bpm', 'bpm_changes', 'total_notes']].head(10))
df.to_csv('taiko_features.csv', index=False)

Processed 17 courses


KeyError: "['initial_bpm'] not in index"